# 🧪 Laboratório 3 — **Tools** com Multiagentes: Auditoria de Contratos Automática (CrewAI)

**Objetivo do Lab:** aprender a integrar **Tools** para que os agentes **ajam no mundo** (ler arquivos reais) e **entreguem análise de risco**.

> Pipeline (sequencial):  
> **Agente 1 (Leitor de Contrato)** usa uma **Tool** para ler o arquivo →  
> **Agente 2 (Analista Jurídico)** interpreta o conteúdo e gera o diagnóstico.


## 🔑 LLM (Groq) — configuração rápida (OpenAI-compatible)

1. Crie sua conta e gere uma chave em: https://console.groq.com  
2. No Colab, defina **variáveis de ambiente** do provedor compatível:
   - `OPENAI_API_KEY`: sua chave da Groq  
   - `OPENAI_API_BASE`: endpoint → `https://api.groq.com/openai/v1`

> Com isso, os agentes/tarefas do CrewAI usarão a Groq automaticamente.


## 🔧 Passo 0 — Instalar dependências
Se o Colab pedir para reiniciar, aceite e depois rode novamente esta célula.


In [ ]:
!pip -q install crewai crewai-tools langchain python-dotenv pypdf


## 🔐 Passo 1 — Configurar chaves no ambiente (Groq ou outro provedor compatível)

> Opcionalmente, cole suas chaves abaixo (somente para este notebook).  
> Em produção, prefira **segredos** e **variáveis de ambiente** seguras.


In [ ]:
import os

# ===> DESCOMENTE e COLE suas chaves para usar a Groq (OpenAI-compatible) <===
# os.environ["OPENAI_API_KEY"] = "SUA_CHAVE_DA_GROQ_AQUI"
# os.environ["OPENAI_API_BASE"] = "https://api.groq.com/openai/v1"

print("OPENAI_API_KEY set?", "OPENAI_API_KEY" in os.environ)
print("OPENAI_API_BASE set?", "OPENAI_API_BASE" in os.environ)


## 🧠 Passo 2 — O que é uma **Tool** no CrewAI?

- Uma **Tool** permite que o agente **execute ações** (ler arquivo, buscar web, chamar API).  
- No CrewAI, configuramos a Tool e **anexamos** ao agente com `tools=[...]`.  
- Neste lab, usaremos a **FileReadTool** (do `crewai-tools`) para ler **texto** de um arquivo.


## 📤 Passo 3 — Faça upload de um **contrato em PDF**

> Use um contrato real (ex.: NDA/Prestação de Serviços).  
> O PDF será convertido para **TXT** para a Tool ler com mais estabilidade.


In [ ]:
from google.colab import files

print("Selecione seu arquivo PDF de contrato...")
uploaded = files.upload()

# Pegamos o primeiro arquivo enviado
pdf_path = None
for k in uploaded.keys():
    if k.lower().endswith(".pdf"):
        pdf_path = k
        break

if pdf_path is None:
    raise ValueError("Nenhum PDF foi enviado. Faça o upload de um arquivo .pdf.")
print("PDF recebido:", pdf_path)


## 🧾 Passo 4 — Converter **PDF → TXT** (pré-processamento)

Algumas ferramentas lidam melhor com texto puro. Vamos extrair o texto do PDF para `/content/contract.txt`.


In [ ]:
from pypdf import PdfReader

txt_path = "/content/contract.txt"
with open(txt_path, "w", encoding="utf-8") as out:
    reader = PdfReader(pdf_path)
    for i, page in enumerate(reader.pages):
        try:
            text = page.extract_text() or ""
        except Exception as e:
            text = ""
        # separador de páginas para ajudar a navegação do agente
        out.write(f"\n\n===== [Página {i+1}] =====\n\n")
        out.write(text)

print("TXT gerado em:", txt_path)
# Exibir uma amostra
with open(txt_path, "r", encoding="utf-8") as f:
    sample = "".join([next(f) for _ in range(60)])
print("Prévia do TXT:\n", sample[:1000], "...")


## ✅ Passo 5 — Imports, Tool e Agentes

Vamos usar a **FileReadTool** para permitir que o agente **leia** o texto do contrato.


In [ ]:
from crewai import Agent, Task, Crew, Process
from crewai_tools import FileReadTool

# Tool apontando para o TXT do contrato
file_tool = FileReadTool(file_path=txt_path)

# 🔹 Agente 1 — Leitor de Contrato (usa a Tool)
reader_agent = Agent(
    role="Leitor de Contrato",
    goal=("Ler e extrair cláusulas e trechos relevantes do contrato, "
          "organizando por tópico e mantendo fidelidade ao texto."),
    backstory=("Você é um analista jurídico metódico. Seu objetivo é encontrar e "
               "apresentar os trechos exatos do contrato que tratam de cada cláusula investigada."),
    tools=[file_tool],
    allow_delegation=False,
    verbose=True
)

# 🔹 Agente 2 — Analista Jurídico (interpreta)
legal_agent = Agent(
    role="Analista Jurídico",
    goal=("Avaliar riscos, lacunas e pontos críticos do contrato a partir dos trechos "
          "fornecidos pelo Leitor de Contrato, sugerindo melhorias de redação."),
    backstory=("Você atua como consultor jurídico. Sua análise é objetiva, prática e orientada a risco."),
    allow_delegation=False,
    verbose=True
)


## 🗂️ Passo 6 — Tarefas (sequenciais)

1. **Leitura & Extração (Tool)** → 2. **Análise & Recomendações**  
**Dica:** seja claro no **expected_output**.


In [ ]:
# 🔸 Task 1 — Leitura & Extração com Tool
extract_task = Task(
    description=(
        "Usando a ferramenta de leitura de arquivos, abra e leia o conteúdo do contrato. "
        "Procure e extraia trechos relacionados aos seguintes tópicos (se existirem): "
        "1) Responsabilidade e Limitação de Responsabilidade; "
        "2) Indenização; "
        "3) Confidencialidade e Propriedade Intelectual; "
        "4) Proteção de Dados (LGPD) e Segurança; "
        "5) SLA/Qualidade de Serviço e Multas; "
        "6) Rescisão/Resolução e Penalidades; "
        "7) Foro/Lei Aplicável; "
        "8) Preço/Reajuste/Correção; "
        "9) Prazos e Entregáveis. "
        "Apresente os trechos **citando página** e entre aspas. "
        "Se algo não for encontrado, registre como 'não localizado'."
    ),
    expected_output=(
        "Documento estruturado com **tabela** ou **lista** de tópicos, incluindo: "
        "tópico, citação do contrato (com página), e observações (se houver)."
    ),
    agent=reader_agent,
)

# 🔸 Task 2 — Análise & Recomendações
analyze_task = Task(
    description=(
        "Com base nos trechos extraídos pelo Leitor de Contrato, elabore um **diagnóstico jurídico** com:\n"
        "• **Resumo executivo** (até 6 linhas)\n"
        "• **Riscos/Red Flags** por tópico\n"
        "• **Sugestões de melhoria de redação** (exemplos curtos, objetivos)\n"
        "• **Perguntas de due diligence** (se faltarem cláusulas críticas)\n"
        "• **Níveis de severidade** (Alta/Média/ Baixa) e **prioridade**\n\n"
        "Se um tópico foi marcado como 'não localizado', indique a recomendação correspondente.\n"
        "Evite jargões excessivos e **não invente texto** que não conste do contrato."
    ),
    expected_output=(
        "Relatório estruturado em Markdown com as seções acima, pronto para enviar ao time jurídico."
    ),
    agent=legal_agent,
    context=[extract_task],  # recebe o output do leitor
)


## 🚀 Passo 7 — Orquestrar em **sequência** e executar


In [ ]:
crew = Crew(
    agents=[reader_agent, legal_agent],
    tasks=[extract_task, analyze_task],
    process=Process.sequential,
    verbose=True
)

print(">>> Executando Lab 3 (Tools → Leitura e Análise)...")
result = crew.kickoff()


## 👀 Passo 8 — Visualizar o relatório final (Markdown)


In [ ]:
from IPython.display import display, Markdown
display(Markdown(result.raw))


## 🔁 Variações e extensões (para praticar)

1) **Checagem de completude**: adicione uma 3ª tarefa ("Revisor de Completude") que verifica se todos os tópicos foram cobertos.  
2) **Fato x Opinião**: peça ao Analista para sinalizar frases opinativas com `(OPINIÃO)` e fatos com `(FATO)`.  
3) **Busca externa (avançado)**: integre `SerperDevTool` para buscar boas práticas contratuais públicas e sugerir cláusulas-modelo.  
4) **Avaliação rápida**: crie um **score de risco** agregado (0–100) e explique como foi composto.  
5) **Multi-idioma**: se o contrato estiver em inglês, peça tradução de trechos críticos para PT-BR.
